In [1]:
import graph_tool as gt
import torch
import pickle
from src.diffusion_model_discrete import DiscreteDenoisingDiffusion
from sample import get_model_sbm
from src.diffusion import diffusion_utils
import networkx as nx
import matplotlib.pyplot as plt
from src.analysis.visualization import NonMolecularVisualization
import torch.nn.functional as F

<frozen importlib._bootstrap>:228: RuntimeWarning: to-Python converter for std::pair<double, double> already registered; second conversion method ignored.


In [2]:
model = get_model_sbm()


(ipykernel_launcher.py:39755): Gtk-WARNING **: 02:15:52.668: Locale not supported by C library.
	Using the fallback 'C' locale.
/home/renyi/anaconda3/envs/digress/lib/python3.9/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Marginal distribution of the classes: tensor([1.]) for nodes, tensor([0.9156, 0.0844]) for edges


In [4]:
G_t = model.sample_G_T()

In [5]:
_, e1 = model.sample_one(G_t)
_, e2 = model.sample_one(G_t)
assert torch.equal(e2, e1)